In [15]:
import requests
import pandas as pd
from mainnet_launch.constants import *
from web3 import Web3


def paginate_query(api_url: str, query: str, variables: dict, data_col: str) -> list[dict]:
    """
    Helper to page through a GraphQL connection using `first`/`skip`.

    :param api_url: GraphQL endpoint
    :param query: The GraphQL query string, expecting $first and $skip variables
    :param variables: Base variables (e.g. {"autoEthAddress": "..."}).
                      first/skip will be merged in each loop.
    :param batch_size: Number of items to fetch per request.
    :return: List of result dicts
    """

    all_records = []
    skip = 0

    while True:
        vars_with_pagination = {**variables, "first": 500, "skip": skip}
        resp = requests.post(api_url, json={"query": query, "variables": vars_with_pagination})
        resp.raise_for_status()
        batch = resp.json()["data"][data_col]

        if not batch:
            break

        all_records.extend(batch)
        skip += 500

    return all_records


def _get_subgraph_api(chain):
    if chain == "eth":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-eth-mainnet/api"
    elif chain == "base":
        api_url = "https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-base-mainnet/api"
    else:
        raise ValueError("bad chain", chain)

    return api_url


def fetch_autopool_rebalance_events_from_subgraph(autopool_eth_addr: str, chain: str) -> list[dict]:
    """
    Fetches all AutopoolRebalances entries for the given autopool.
    """
    subgraph_url = _get_subgraph_api(chain)

    query = """
    query($autoEthAddress: String!, $first: Int!, $skip: Int!) {
      autopoolRebalances(
        first: $first,
        skip: $skip,
        orderBy: id,
        orderDirection: desc,
        where: { autopool: $autoEthAddress }
      ) {
        transactionHash
        timestamp
        blockNumber

        tokenInAmount
        tokenOut {
        id
        decimals
        
        }        
        destinationInAddress

        tokenIn {
        id
        decimals
        
        }   

        tokenOutAmount
        destinationOutAddress
        
      }
    }
    """

    # Adjust `first` as needed; paginate_query will loop over skip increments of `first`
    df = pd.DataFrame.from_records(
        paginate_query(
            subgraph_url,
            query,
            variables={"autoEthAddress": autopool_eth_addr.lower(), "first": 1000, "skip": 0},
            data_col="autopoolRebalances",
        )
    )

    df["tokenInAddress"] = df["tokenIn"].apply(lambda x: x["id"])
    df["tokenOutAddress"] = df["tokenOut"].apply(lambda x: x["id"])

    df["tokenOutAmount"] = df.apply(
        lambda row: int(row["tokenOutAmount"]) / (10 ** int(row["tokenOut"]["decimals"])), axis=1
    )
    df["tokenInAmount"] = df.apply(
        lambda row: int(row["tokenInAmount"]) / (10 ** int(row["tokenIn"]["decimals"])), axis=1
    )

    df["blockNumber"] = df["blockNumber"].astype(int)

    df["destinationInAddress"] = df["destinationInAddress"].apply(lambda x: Web3.toChecksumAddress(x))
    df["tokenInAddress"] = df["tokenInAddress"].apply(lambda x: Web3.toChecksumAddress(x))

    df["destinationOutAddress"] = df["destinationOutAddress"].apply(lambda x: Web3.toChecksumAddress(x))
    df["tokenOutAddress"] = df["tokenOutAddress"].apply(lambda x: Web3.toChecksumAddress(x))
    df = df.sort_values("blockNumber")
    return df


rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph(AUTO_USD.autopool_eth_addr, "eth")
rebalance_event_df

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenOut,destinationInAddress,tokenIn,tokenOutAmount,destinationOutAddress,tokenInAddress,tokenOutAddress
218,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,1743708611,22190481,1.000087e+02,{'id': '0x9d39a5de30e57443bff2a8307a4256c8797a...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,8.589984e+01,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
250,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,1743723995,22191759,9.995465e+01,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,9.832635e+01,0x6F628dcCD275feA4277722D177265741031E09d7,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4
348,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,1743782255,22196597,2.534899e+04,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,2.550339e+04,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
235,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,1743786695,22196965,1.880682e+04,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,1.912754e+04,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
158,0x8ac67f7b06d5dcaa032ce596da03410177deaac9028a...,1743793139,22197502,1.426166e+04,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,1.434566e+04,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
...,...,...,...,...,...,...,...,...,...,...,...
311,0x2a6165a5a84c8092203244c2097a419c7b890cc62263...,1747844303,22532409,1.226160e+06,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7876F91BB22148345b3De16af9448081E9853830,{'id': '0x9fb7b4477576fe5b32be4c1843afb1e55f25...,1.409821e+06,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
8,0xfad89b262fa28252edc915280cd333b73b71e1235ac0...,1747858619,22533589,5.646532e+05,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7876F91BB22148345b3De16af9448081E9853830,{'id': '0x9fb7b4477576fe5b32be4c1843afb1e55f25...,6.492902e+05,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
107,0xae2e87f39bd139022c8042ecada5a50aa7cf3de6e0e0...,1747876619,22535080,2.823220e+05,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7876F91BB22148345b3De16af9448081E9853830,{'id': '0x9fb7b4477576fe5b32be4c1843afb1e55f25...,3.246451e+05,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
375,0x056c10e77731b350d783e2e2ff96328f02a48178e75f...,1747881491,22535484,1.411604e+05,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7876F91BB22148345b3De16af9448081E9853830,{'id': '0x9fb7b4477576fe5b32be4c1843afb1e55f25...,1.623226e+05,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48


In [34]:
few_rebalance_events = (
    rebalance_event_df[
        [
            "transactionHash",
            "blockNumber",
            "tokenInAddress",
            "tokenInAmount",
            "destinationInAddress",
            "tokenOutAddress",
            "tokenOutAmount",
            "destinationOutAddress",
        ]
    ]
    .tail(1)
    .T
)
few_rebalance_events

,65
transactionHash,0xd11138a0694bf9c31415ab4d445914cb4d1d01804aaa...
blockNumber,22537762
tokenInAddress,0x5C20B550819128074FD538Edf79791733ccEdd18
tokenInAmount,61127.128499
destinationInAddress,0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61
tokenOutAddress,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
tokenOutAmount,70000.0
destinationOutAddress,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35


In [35]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_states_table import (
    build_lp_token_spot_price_calls,
)
from mainnet_launch.constants import AUTO_USD
from multicall import Call

from mainnet_launch.data_fetching.get_state_by_block import *

calls = build_lp_token_spot_price_calls(
    destination_addresses=["0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61", "0xa7569A44f348d3D70d8ad5889e50F78E33d80D35"],
    lp_token_addresses=["0x5C20B550819128074FD538Edf79791733ccEdd18", "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"],
    pool_addresses=["0x5C20B550819128074FD538Edf79791733ccEdd18", "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"],
    chain=ETH_CHAIN,
    base_asset=AUTO_USD.base_asset,
)

# 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48 == 1 is autoUSD
state = get_state_by_one_block(calls, 22537762, ETH_CHAIN)
state

{('0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61',
  'lp_token_spot_and_safe'): (1.145978, 1.145975),
 ('0xa7569A44f348d3D70d8ad5889e50F78E33d80D35',
  'lp_token_spot_and_safe'): None}

In [38]:
autoUSD_flash_borrow_solver = "0xD02b50CFc6c2903bF13638B28D081ad11515B6f9"
usdt = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
flash_borrow_usdt_bal = Call(
    usdt,
    ["balanceOf(address)(uint256)", autoUSD_flash_borrow_solver],
    [(("flash_borrow_usdt_balance"), safe_normalize_6_with_bool_success)],
)
before = get_state_by_one_block([flash_borrow_usdt_bal], 22537762 - 1, ETH_CHAIN)
after = get_state_by_one_block([flash_borrow_usdt_bal], 22537762, ETH_CHAIN)
before, after

({'flash_borrow_usdt_balance': 61.051629}, {'flash_borrow_usdt_balance': 0.0})

In [40]:
fUSDT_less = 61.051629 / 1.145978

fUSDT_less

53.27469550026266

In [43]:
70_000 - (61127.128499 - fUSDT_less) * 1.145978

10.707165972984512

In [45]:
69961501847 / 1e6

69961.501847

In [48]:
10000 * 832381825491494 / 1e18

8.32381825491494

In [44]:
10_000 * 10.707165972984512 / 70_000

1.5295951389977875

In [31]:
61_127.128499 * 1.145978

70050.34446302701

In [30]:
61_127.128499 * 1.145975

70050.16108164152

In [29]:
61_127.128499 * 1.145978

70050.34446302701

In [ ]:
rebalance_event_df

In [ ]:
from mainnet_launch.database.schema.full import (
    Blocks,
    DestinationTokenValues,
    DestinationStates,
    Destinations,
    AutopoolStates,
    AutopoolDestinations
)
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
)
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_states_table import 



spot_price_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            DestinationTokenValues,
            [
                DestinationTokenValues.destination_vault_address,
                DestinationTokenValues.spot_price,
                DestinationTokenValues.block
            ],
        ),        
        TableSelector(
            Destinations,
            [
                Destinations.underlying_name,
                Destinations.underlying,
            ],
            join_on=DestinationTokenValues.destination_vault_address == Destinations.destination_vault_address
        ),
    ]

 )
spot_price_df

2025-05-22 10:24:30,641 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 10:24:30,641 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-22 10:24:30,641 INFO sqlalchemy.engine.Engine [cached since 41.73s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x16a3ba9e0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-22 10:24:30,642 INFO sqlalchemy.engine.Engine SELECT
    destination_token_values.destination_vault_address,
    destination_token_values.spot_price,
    destination_token_v

,destination_vault_address,spot_price,block,underlying_name,underlying
0,0x734FE415357c9c585794892405dCD714c5Ae68c5,1.127298,22523468,Staked FRAX,0xA663B02CF0a4b149d2aD41910CB81e23e1c41c32
1,0xa345ceECCF8fe6aE33fe1D655B4806492251c2A8,1.000135,22506551,GHO/crvUSD,0x635EF0056A597D13863B73825CcA297236578595
2,0xd7900d87069C815a299bdA7aFDcd7eEe98fe4b6c,0.999146,22458909,Curve Strategic Ethena Reserves,0x5B03CcCAb7BA3010fA5CAd23746cbf0794938e96
3,0xa345ceECCF8fe6aE33fe1D655B4806492251c2A8,1.000210,22221247,GHO/crvUSD,0x635EF0056A597D13863B73825CcA297236578595
4,0x734FE415357c9c585794892405dCD714c5Ae68c5,1.127234,22456766,Staked FRAX,0xA663B02CF0a4b149d2aD41910CB81e23e1c41c32
...,...,...,...,...,...
53460,0xA94031Ed4b316B043464FDd5482877F42A39845a,0.998955,30541326,Volatile AMM - WETH/wrsETH,0xA24382874A6FD59de45BbccFa160488647514c28
53461,0xb3723E69D6e4553239d7dAf514bF8ff7998500E0,0.999607,30541326,Balancer weETH/wETH,0xaB99a3e856dEb448eD99713dfce62F937E2d4D74
53462,0xAADf01DD90aE0A6Bb9Eb908294658037096E0404,1.000000,30541326,baseETH,0xAADf01DD90aE0A6Bb9Eb908294658037096E0404
53463,0xb3723E69D6e4553239d7dAf514bF8ff7998500E0,1.068021,30541326,Balancer weETH/wETH,0xaB99a3e856dEb448eD99713dfce62F937E2d4D74


In [12]:
pd.merge(
    rebalance_event_df,
    spot_price_df,
    left_on=["blockNumber", "destinationInAddress"],
    right_on=["block", "destination_vault_address"],
    how="left",
)

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenOut,destinationInAddress,tokenIn,tokenOutAmount,destinationOutAddress,tokenInAddress,tokenOutAddress,destination_vault_address,spot_price,block,underlying_name,underlying
0,0xff9eefa3b3437e6abac7729bce32f32dba471cf03dbd...,1747039487,22466103,99165.054265,{'id': '0x5c20b550819128074fd538edf79791733cce...,0xb949C40F18fB52747d96850696c954718168fde3,{'id': '0x81a2612f6dea269a6dd1f6deab45c5424ee2...,87333.346142,0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61,0x81A2612F6dEA269a6Dd1F6DeAb45C5424EE2c4b7,0x5C20B550819128074FD538Edf79791733ccEdd18,NaN,NaN,NaN,NaN,NaN
1,0xff8915d0bd9054e7678c4366ae8fd5a0a3cce80943be...,1744988327,22296632,148881.150087,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,NaN,NaN,NaN,NaN,NaN
2,0xff3d22a3564c8121f28d7d47feffb8251785ad2c48e4...,1744997471,22297389,148885.474474,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,NaN,NaN,NaN,NaN,NaN
3,0xfecf659a8d1c06c45010afc47fcefc6405d906a85e84...,1746235043,22399927,320651.575857,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xC5c95fCad37E466E25e6ecA1977bbF75C0E1004a,{'id': '0xbeef01735c132ada46aa9aa4c54623caa92a...,347222.682150,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xBEEF01735c132Ada46AA9aA4c54623cAA92A64CB,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,NaN,NaN,NaN,NaN,NaN
4,0xfd0bbd018b24792dbce75f17221abb20609cd084012f...,1746819107,22447978,99154.036813,{'id': '0x5b03cccab7ba3010fa5cad23746cbf079493...,0xb949C40F18fB52747d96850696c954718168fde3,{'id': '0x81a2612f6dea269a6dd1f6deab45c5424ee2...,99250.165003,0xd7900d87069C815a299bdA7aFDcd7eEe98fe4b6c,0x81A2612F6dEA269a6Dd1F6DeAb45C5424EE2c4b7,0x5B03CcCAb7BA3010fA5CAd23746cbf0794938e96,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0x02a5528289efe28718bed54e9c5b0772f16eba69d609...,1744695875,22272381,76347.504659,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,77679.210200,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,NaN,NaN,NaN,NaN,NaN
384,0x02065d1b16c7c693afe6358f38647b921a095d6876ec...,1745911055,22373171,26611.724622,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0x7583b1589aDD33320366A48A92794D77763FAE9e,{'id': '0x390f3595bca2df7d23783dfd126427cceb99...,27068.157900,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,NaN,NaN,NaN,NaN,NaN
385,0x012e26a2633c10388ffc3ee1a2a95c9569b54ff4f918...,1744556327,22260800,140053.102440,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,0xC099899d0278CE83976218Cbe58D01dD382dcA32,{'id': '0x57064f49ad7123c92560882a45518374ad98...,149036.479162,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0x57064F49Ad7123C92560882a45518374ad982e85,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,NaN,NaN,NaN,NaN,NaN
386,0x00ff23899cd9bb0075ebe5ac23a3faa815088ddd32ab...,1744906595,22289850,148973.430940,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,{'id': '0x4f493b7de8aac7d55f71853688b1f7c8f024...,150000.000000,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,NaN,NaN,NaN,NaN,NaN


In [7]:
from mainnet_launch.database.schema.full import (
    Blocks,
    DestinationTokenValues,
    DestinationStates,
    Destinations,
    AutopoolStates,
    AutopoolDestinations,
)
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
)
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_states_table import (
    ensure_destination_states_are_current,
)


autopool_and_destinations_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            AutopoolDestinations,
            [
                AutopoolDestinations.destination_vault_address,
                AutopoolDestinations.autopool_vault_address,
            ],
        ),
        TableSelector(
            Destinations,
            Destinations.underlying_name,
            join_on=AutopoolDestinations.destination_vault_address == Destinations.destination_vault_address,
        ),
    ],
    where_clause=(Destinations.chain_id == ETH_CHAIN.chain_id)
    & (AutopoolDestinations.autopool_vault_address.in_([a.autopool_eth_addr for a in [AUTO_USD]])),
)

autopool_to_all_ever_active_destinations = (
    autopool_and_destinations_df.groupby("autopool_vault_address")["destination_vault_address"].apply(tuple).to_dict()
)


# (lp_token_spot_price, lp_token_safe_price)
lp_safe_and_spot_price_df = _fetch_lp_token_spot_prices(
    autopool_to_all_ever_active_destinations, [int(b) for b in rebalance_event_df["blockNumber"]], ETH_CHAIN
)
lp_safe_and_spot_price_df

2025-05-22 10:24:32,000 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 10:24:32,000 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-22 10:24:32,000 INFO sqlalchemy.engine.Engine [cached since 43.09s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x16a3ba7a0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-22 10:24:32,000 INFO sqlalchemy.engine.Engine SELECT
    autopool_destinations.destination_vault_address,
    autopool_destinations.autopool_vault_address,
    destinations.

NameError: name '_fetch_lp_token_spot_prices' is not defined

In [ ]:
lp_token_spot_price = lp_safe_and_spot_price_df.map(lambda x: x[0] if isinstance(x, tuple) else None)
lp_token_spot_price["block"] = lp_safe_and_spot_price_df["block"]
lp_token_safe_price = lp_safe_and_spot_price_df.map(lambda x: x[1] if isinstance(x, tuple) else None)
lp_token_safe_price["block"] = lp_safe_and_spot_price_df["block"]

In [ ]:
spot_long = lp_token_spot_price.melt(id_vars=["block"], var_name="lp_token_address", value_name="value")
spot_long["lp_token_address"] = spot_long["lp_token_address"].apply(lambda x: x[0])

safe_long = lp_token_safe_price.melt(id_vars=["block"], var_name="lp_token_address", value_name="value")
safe_long["lp_token_address"] = safe_long["lp_token_address"].apply(lambda x: x[0])

In [ ]:
block_token_to_safe_price[(22532409, "0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33".lower())]

KeyError: (22532409, '0x9fb7b4477576fe5b32be4c1843afb1e55f251b33')

In [ ]:
df1.isna().sum()

In [ ]:
import plotly.express as px

px.bar(df1.resample("1d")[["swap_cost"]].sum(), y="swap_cost", title="Swap Cost")

In [ ]:
avg_nav = df1.resample("1d")["total_nav"].mean()
total_swap_cost = df1.resample("1d")["swap_cost"].sum()
total_swap_cost

In [ ]:
avg_nav = df1.resample("1d")["total_nav"].mean() * 1e12
total_swap_cost = df1.resample("1d")["swap_cost"].sum()
import plotly.express as px

percent_of_swap_cost_per_day = 100 * (total_swap_cost) / avg_nav
px.bar(percent_of_swap_cost_per_day)

In [ ]:
df1.tail(1).T

In [ ]:
df1.index = pd.to_datetime(df1["timestamp"].astype(int), unit="s", utc=True)

df1.sort_index().to_csv("autoUSD_slippage_with_nav.csv")

In [ ]:

import plotly .express as px



df1.sort_values(')

In [ ]:
df1[
    [
        "transactionHash",
        "timestamp",
        "blockNumber",
        "tokenInAmount",
        "destinationInAddress",
        "tokenOutAmount",
        "destinationOutAddress",
        "tokenInAddress",
        "tokenOutAddress",
        "lp_token_in_spot_price",
        "lp_token_out_spot_price",
        "spot_value_in",
        "spot_value_out",
        "slippage",
    ]
].to_csv("autoUSD_slippage.csv")

In [ ]:
df1["slippage"] < 0

In [ ]:
df1["slippage"]

In [ ]:
px.histogram(df1["swap_cost"], title="autoUSD slippage")

In [ ]:
destination_states_df

In [ ]:
destination_states_df[destination_states_df["underlying"] == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"]

In [ ]:
df1["tokenOutAddress"].value_counts()

In [ ]:
destination_states_df

In [ ]:
rebalance_event_df

In [ ]:
rebalance_event_df[["blockNumber", "destinationInAddress", "tokenInAddress"]]

In [ ]:
destination_states_df

In [ ]:
destination_states_df[["block", "destination_vault_address", "underlying"]]

In [ ]:
destination_states_df = destination_states_df.drop_duplicates()

In [ ]:
rebalance_event_df.head(1).T

In [ ]:
# 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48

In [ ]:
token_value_df[
    (token_value_df["block"] == 22296632)
    & (token_value_df["destination_vault_address"] == "0xa7569A44f348d3D70d8ad5889e50F78E33d80D35")
    & (token_value_df["token_address"] == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48")
]

In [ ]:
rebalance_event_df

In [ ]:
destination_states_df["underlying_symbol"].value_counts()